In [13]:
# werkzeug
# shapely=1.7.1
# geopandas
# pandas
# contextily
# matplotlib
# docxtpl
# docx

In [1]:
from werkzeug.wrappers import Request, Response
from jinja2 import Template

from datetime import datetime, timedelta

import collections
from amigocloud import AmigoCloud

from datetime import datetime

from shapely import wkb
import geopandas
import pandas as pd
import contextily as ctx
import matplotlib.pyplot as plt

from docxtpl import DocxTemplate
import docxtpl
from docx.shared import Mm
import requests

def convertir_wkb(wkb_data):
    return wkb.loads(wkb_data, hex=True)[0]

C:\Users\Bismar\.conda\envs\utea_reportes\lib\importlib\_bootstrap.py:219: RuntimeWarning: greenlet.greenlet size changed, may indicate binary incompatibility. Expected 144 from C header, got 152 from PyObject
  return f(*args, **kwds)
C:\Users\Bismar\.conda\envs\utea_reportes\lib\importlib\_bootstrap.py:219: RuntimeWarning: greenlet.greenlet size changed, may indicate binary incompatibility. Expected 144 from C header, got 152 from PyObject
  return f(*args, **kwds)
C:\Users\Bismar\.conda\envs\utea_reportes\lib\importlib\_bootstrap.py:219: RuntimeWarning: greenlet.greenlet size changed, may indicate binary incompatibility. Expected 144 from C header, got 152 from PyObject
  return f(*args, **kwds)
C:\Users\Bismar\.conda\envs\utea_reportes\lib\importlib\_bootstrap.py:219: RuntimeWarning: greenlet.greenlet size changed, may indicate binary incompatibility. Expected 144 from C header, got 152 from PyObject
  return f(*args, **kwds)


In [2]:
# CONECTAR A AMIGOCLOUD
acceso = open('../api_key/api.key','r')
api_token = acceso.readlines()[0]
acceso.close()
amigocloud = AmigoCloud(token=api_token)
amigocloud

In [3]:
def generar_reporte(codigo_canhero):
    # EXTRAER DATOS DE CREDITO
    query_credito = {'query': 'select * from dataset_307560 where cast(SPLIT_PART(canhero, \'/\', 1) as integer) = {cod_ca}'.format(cod_ca=codigo_canhero)}
    credito_select = amigocloud.get('https://app.amigocloud.com/api/v1/projects/32672/sql', query_credito)
    credito_data = credito_select['data'][0]
    credito_data['fecha_informe'] = (datetime.strptime(credito_data['fecha_1ra_insp'].split(' ')[0], '%Y-%m-%d') + timedelta(days=1)).strftime('%d/%m/%Y')
    credito_data['fecha_1ra_insp'] = datetime.strptime(credito_data['fecha_1ra_insp'].split(' ')[0], '%Y-%m-%d').strftime('%d/%m/%Y')
    credito = collections.namedtuple("credito", credito_data.keys())(*credito_data.values())

    # EXTRAER DATOS DE PROPIEDADES
    query_propiedades = {'query': 'select * from dataset_307562 where insp_ref_prop = \'{insp_ref_prop}\''.format(insp_ref_prop=credito.amigo_id)}
    propiedades_select = amigocloud.get('https://app.amigocloud.com/api/v1/projects/32672/sql', query_propiedades)
    propiedades_data = propiedades_select['data']

    # EXTRAER DATOS DE LOTES
    propiedades = []
    for prop in propiedades_data:
        prop['lotes'] = []
        query_lotes = {'query': 'select * from dataset_307563 where prop_ref_lote=\'{prop_ref_lote}\''.format(prop_ref_lote=prop['amigo_id'])}
        lotes_select = amigocloud.get('https://app.amigocloud.com/api/v1/projects/32672/sql', query_lotes)
        for lote_select in lotes_select['data']:
            lote = collections.namedtuple("lote", lote_select.keys())(*lote_select.values())
            prop['lotes'].append(lote)
        propiedades.append(collections.namedtuple("propiedad", prop.keys())(*prop.values()))

    # EXTRAER FOTOS
    fotos = []
    for propiedad in propiedades:
        for lote in propiedad.lotes:
            print(lote.lote, lote.amigo_id)
            query_fotos = {'query': 'select source_amigo_id, s3_filename from gallery_46845 where source_amigo_id=\'{source_amigo_id}\''.format(source_amigo_id=lote.amigo_id)}
            fotos_filename = amigocloud.get('https://app.amigocloud.com/api/v1/projects/32672/sql', query_fotos)
            if fotos_filename['data'] != []:
                dic = {'lote': lote.lote, 'file_name': fotos_filename['data'][0]['s3_filename']}
                fotos.append(dic)

    # GENERERAR PLANOS
    i = 0
    lista_planos = []
    for propiedad in propiedades:
        i = i + 1
        lista_planos.append(generar_plano(propiedad, i))


    # GENERAR REPORTE
    doc = DocxTemplate("templates/template_word_1ra.docx")
    #generar lista de InlineImage de planos 
    lista_InlineImage = []
    for plano in lista_planos:
        lista_InlineImage.append(docxtpl.InlineImage(doc, image_descriptor=plano, width=Mm(150)))
    #descargar fotos y generar lista InlineImage
    lista_fotos_inline = []
    for foto in fotos:
        #response = requests.get("https://www-amigocloud.s3.amazonaws.com/gallery/" + foto['file_name'])
        response = requests.get("https://www-amigocloud.s3.amazonaws.com/gallery/" + foto['file_name'])
        file = open('fotos/' + foto['file_name'], "wb")
        file.write(response.content)
        file.close()
        lista_fotos_inline.append({'foto': docxtpl.InlineImage(doc, image_descriptor='fotos/' + foto['file_name'], width=Mm(120)), 'lote': foto['lote']})
    firma_respon = None
    if credito.respon_1ra_insp == 'Rogelio Acuña Rodríguez':
        firma_respon = docxtpl.InlineImage(doc, image_descriptor='templates/firma_rogelio.png', width=Mm(60))
    else:
        firma_respon = docxtpl.InlineImage(doc, image_descriptor='templates/firma_jaldin.png', width=Mm(60))
    context = {'credito':credito, 'propiedades':propiedades, 'planos':lista_InlineImage, 'fotos':lista_fotos_inline, 'firma':firma_respon}
    doc.render(context)
    file_name = credito.canhero.replace(' / ', '_')
    doc.save('_' + file_name + '.docx')

In [4]:
def generar_plano(propiedad, i):
    lotes_lista = []
    for lote in propiedad.lotes:
        lotes_lista.append(lote._asdict())
    df = pd.DataFrame(lotes_lista)
    df['geometria'] = df['geometria'].apply(convertir_wkb)

    #Convertir a GeoDataFrame
    data = geopandas.GeoDataFrame(df, geometry='geometria')
    #data
    #ctx.add_basemap(ax, source=ctx.providers.Esri.WorldImagery)

    data['coords'] = data['geometria'].apply(lambda x: x.representative_point().coords[:])
    data['coords'] = [coords[0] for coords in data['coords']]

    data.crs = "EPSG:4326"
    data = data.to_crs(epsg=3857)

    
    fig = plt.figure(i, figsize=(20,20))
    #fig = plt.figure(1)
    ax = None
    ax = fig.add_subplot()

    #data.set_crs(epsg=4326, inplace=True)

    data.apply(lambda x: ax.annotate(text=x.lote + ' \n' + str(x.superficie) + ' ha', xy=x.geometria.centroid.coords[0], ha='center', va='center', color='black', fontsize=12, weight=1000, bbox=dict(facecolor=(1,1,1,0.3), edgecolor='none', pad=0)), axis=1);

    #data.boundary.plot(ax=ax, color=(0,0,0,0), linewidth=0)

    #data = data.to_crs(epsg=3857)

    minx, miny, maxx, maxy = data.total_bounds
    ax.set_xlim(minx - 500, maxx + 500)
    ax.set_ylim(miny - 400, maxy + 400)

    data.plot(ax=ax, edgecolor='r', facecolor=(0,0,0,0), linewidth=2, figsize=(20,20))
    #data.plot(ax=ax, edgecolor='r', facecolor=(0,0,0,0), linewidth=2)
    
    ctx.add_basemap(ax, source=ctx.providers.Esri.WorldImagery)
    ax.set_axis_off()
    ax.set_title(str(propiedad.propiedad), fontsize=20)
    ruta = 'planos/' + str(propiedad.amigo_id) + '.jpeg'
    fig.savefig(ruta, dpi = 300, bbox_inches='tight')
    plt.clf()
    return ruta

In [5]:
semana = 'S07'
query = {'query': 'select split_part(canhero, \' / \', 1) cod, split_part(canhero, \' / \', 2) nom from dataset_307560 where semana_1ra = \'{sem}\''.format(sem=semana)}
project = 'https://app.amigocloud.com/api/v1/projects/32672/sql'
select = amigocloud.get(project, query)
data = select['data']

In [11]:
data

[{'cod': '42164', 'nom': 'GUTIERREZ ROJAS ALEXANDER'},
 {'cod': '40144', 'nom': 'RIVERO AGUAYO FRANCISCO '},
 {'cod': '7494', 'nom': 'LINARES S. AIDEE'},
 {'cod': '18028', 'nom': 'RIVERO GUTIERREZ FRANCISCO'},
 {'cod': '41821', 'nom': 'AGROMETALDIBRI S.R.L.'},
 {'cod': '2565', 'nom': 'CANDIA RUIZ JOSE LUIS'},
 {'cod': '5385', 'nom': 'HUANCA CANO MARIA '},
 {'cod': '9722', 'nom': 'OLGUIN FLORES MIGUELINA '},
 {'cod': '41544', 'nom': 'AMURRIO TAPIA JOSE LUIS'},
 {'cod': '41823', 'nom': 'VARGAS MAMANI JOSUE'},
 {'cod': '8028', 'nom': 'ZABALA GARRIDO MARCELO'},
 {'cod': '1218', 'nom': 'BALCAZAR ALCANTARA JOSE LOLY'},
 {'cod': '446', 'nom': 'AGUILERA ANEZ JHONNY'},
 {'cod': '40256', 'nom': 'INSERVA DAMIANA S.R.L.'},
 {'cod': '42025', 'nom': 'TRUJILLO CONDORI JAIME '},
 {'cod': '5345', 'nom': 'HUANCA SUAREZ EDGAR '},
 {'cod': '8140', 'nom': 'MASAI CHAVEZ SIRLE YOBANA '},
 {'cod': '765', 'nom': 'AGROP. CATALA S.R.L.'},
 {'cod': '41546', 'nom': 'CASANOVA BASTOS PEDRO RAMIRO'},
 {'cod': '11227'

In [12]:
#lista = [1068, 18062, 10739, 41955, 2392, 15162, 2517, 8706, 15085, 430, 874, 5498, 2505, 2274]
lista = [int(i['cod']) for i in data]

In [13]:
print(len(lista), 'informes')

21 informes


In [14]:
lista 

[42164,
 40144,
 7494,
 18028,
 41821,
 2565,
 5385,
 9722,
 41544,
 41823,
 8028,
 1218,
 446,
 40256,
 42025,
 5345,
 8140,
 765,
 41546,
 11227,
 41554]

In [15]:
for i in lista:
    generar_reporte(i)
    print('======================', i)

L1 5241404dbe9f45b08d1cc3899eb54076
====================== 42164
L1.1 2e9425d108a4407da4dd82434a483694
====================== 40144
L5 7f528d432da7437599777cfdb8d5878f
L18.3 250705a3e18c4041bc70d75d4e959ef2
L18.2 c50416f03efa4f339f7b72964ff30b43
====================== 7494
L5 dced4c088fee41ebad142d5df600b103
L1.2 35206a63a45f407cb9ed860e9efdd7d8
====================== 18028
L1 3bcc76d194d542aabbbe7994ad84b0f8
====================== 41821
L9 c13ecb5ebcbf4462b5b956cc133de4ff
L8 20221412696c43acba562ab2bf67cdad
L7.2 9db1c9bf9844457293faf5c8c8f8b360
L4.1 0f4d5153045f4ee386e1e880ab0034d5
L5.2 028512cee20341b88b607ef044edabbb
====================== 2565
L1.2 0ba7687fbcb94af69e5d0bdbded1b4ad
L1.2 08d1423c2e3e4bdea7562e072de25200
L2.3 7cef00000dac434aaf29ce9fa61c55b4
====================== 5385
L1 99eef9b868564a698e53241737719cee
L5 9da70bf95b3f47aeb6254c2805c30d35
L6 f5790f35656044dd95225094e315d681
L1.2 8ba654351744495187a44c5e3dc0c0b6
====================== 9722
L1 4ccb25aba50c4b56a24c7c4e5

<Figure size 1440x1440 with 0 Axes>

<Figure size 1440x1440 with 0 Axes>

<Figure size 1440x1440 with 0 Axes>